In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# import cvxopt
data=pd.read_csv('KDDTrain+.txt')
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted'
    ,'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate'
    ,'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
    'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','level'])
data.columns = columns

data['attack_num'] = pd.Categorical(data['attack']).codes
data = data.drop(columns='attack')
#
y=data["attack_num"]

def LabelEncoding(data):
    for col in data.columns:
        if data[col].dtype == 'object':
                label_encoder = LabelEncoder()
                data[col] = label_encoder.fit_transform(data[col])
X=data.drop("attack_num",axis=1)
LabelEncoding(X)
X=X.values
# LabelEncoding(y)
y=y.values

X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.15, shuffle=True)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit the scaler on training data and transform both train and test data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
class SVM:
    def __init__(self, C, learning_rate, num_iter):
        self.learning_rate = learning_rate
        self.num_iter = num_iter
        self.C  = C
        self.weights = None
        self.bias = 0
        self.losses = []
    
    def loss(self, y, y_hat):
        diff = 1-y*y_hat
        diff = np.where(diff < 0, 0, diff)
        loss = self.C*np.sum(diff)/len(y) + np.sum(self.weights**2)/2
        return loss
    
    def fit(self, X, y):
        num_inputs, num_feat = X.shape
        self.weights = np.zeros(num_feat)
        
        for iter in range(self.num_iter):
            pred = np.dot(X, self.weights) + self.bias
            grad_weights = (self.weights - (self.C*np.multiply(X.T,y)).T).T
            
            for weight in range(self.weights.shape[0]):
                grad_weights[weight] = np.where((1-pred) <= 0, self.weights[weight], grad_weights[weight])
            
            grad_weights = np.sum(grad_weights, axis=1)
            self.weights = self.weights - self.learning_rate*grad_weights/num_inputs
            
            grad_bias = -y*self.bias
            grad_bias = np.where((1-pred)<=0, self.bias, grad_bias)
            grad_bias = np.sum(grad_bias, axis=0)
            self.bias = self.bias - self.learning_rate*grad_bias/num_inputs
            self.losses.append(self.loss(y,pred))
    
    def decision(self, X):
        return (np.dot(X, self.weights) + self.bias)
    
    def predict(self, X):
        pred = np.where( self.decision(X)< 0, -1, 1)
        return pred

In [ ]:
class MultiClassSVM:
    def __init__(self, C, learning_rate, num_iter):
        self.C = C
        self.learning_rate = learning_rate
        self.num_iter = num_iter
        self.models = {}
        
    def fit(self, X, y):
        self.classes = np.unique(y)
        for eachClass in  self.classes:
            y_binary = np.where(y==eachClass, 1, -1)
            model = SVM(self.C, self.learning_rate, self.num_iter)
            model.fit(X, y_binary)
            self.models[eachClass] = model
    
    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.classes)))
        for i, c in enumerate(self.classes):
            predictions[:, i] = self.models[c].decision(X)
        return self.classes[np.argmax(predictions, axis=1)]

In [ ]:
clf = MultiClassSVM(C=0.5, learning_rate=0.008, num_iter=300)
clf.fit(X=X_train, y=y_train)
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score
print("Accuracy : ", accuracy_score(y_test, y_pred))
print("precision : ", precision_score(y_test, y_pred, average="macro",zero_division=np.nan))

Accuracy :  0.3329276037256562
precision :  0.4342474524538304


In [41]:
acc = []
prec = []
for C in [0.01, 0.05, 0.1, 0.5, 1]:
    for learning_rate in np.arange(0.005, 0.046, 0.01):
        print(learning_rate)
        clf = MultiClassSVM(C=C, learning_rate=learning_rate, num_iter=100)
        clf.fit(X=X_train, y=y_train)
        y_pred = clf.predict(X_test)
        acc.append(accuracy_score(y_test, y_pred))
        prec.append(precision_score(y_test, y_pred, average="macro",zero_division=np.nan))

0.005
0.015
0.024999999999999998
0.034999999999999996
0.04499999999999999
0.005
0.015
0.024999999999999998
0.034999999999999996
0.04499999999999999
0.005
0.015
0.024999999999999998
0.034999999999999996
0.04499999999999999
0.005
0.015
0.024999999999999998
0.034999999999999996
0.04499999999999999
0.005
0.015
0.024999999999999998
0.034999999999999996
0.04499999999999999


In [42]:
import plotly.graph_objects as go
x_val = [0.01, 0.05, 0.1, 0.5, 1]
y_val = [0.005, 0.015, 0.025, 0.035, 0.045]
fig = go.Figure(data=[go.Scatter3d(
    x=x_val, y=y_val, z=acc,
    mode='markers',
    marker=dict(
        size=5,
        color=acc,            # Set color to the z values
        colorscale='Viridis'
    )
)])

fig.update_layout(scene=dict(
                    xaxis_title='regularization variable C',
                    yaxis_title='learning rate',
                    zaxis_title='accuracy'))

fig.show()

In [44]:
fig = go.Figure(data=[go.Scatter3d(
    x=x_val, y=y_val, z=prec,
    mode='markers',
    marker=dict(
        size=5,
        color=prec,            # Set color to the z values
        colorscale='Viridis'
    )
)])

fig.update_layout(scene=dict(
                    xaxis_title='regularization variable C',
                    yaxis_title='learning rate',
                    zaxis_title='precision'))

fig.show()

**The Accuracy is very low, it might be because of the non-linearity, so implementing the kernel trick**


**Manually implementing the kernel trick needs to create a kernel of size num_inputs x num_inputs, as the data is very big, our machines are not allocating the array, so using the library functions**

In [ ]:


def linear(x, z):
    return np.dot(x, z.T)

class SVM:
    def __init__(self, kernel=linear, C=1):
        self.kernel = kernel
        self.C = C

    def fit(self, X, y):
        self.y = y
        self.X = X
        m, n = X.shape

        self.K = np.zeros((m, m))
        for i in range(m):
            self.K[i, :] = self.kernel(X[i, np.newaxis], self.X)
        P = cvxopt.matrix(np.outer(y, y) * self.K)
        q = cvxopt.matrix(-np.ones((m, 1)))
        G = cvxopt.matrix(np.vstack((np.eye(m) * -1, np.eye(m))))
        h = cvxopt.matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        A = cvxopt.matrix(y, (1, m), "d")
        b = cvxopt.matrix(np.zeros(1))
        cvxopt.solvers.options["show_progress"] = False
        sol = cvxopt.solvers.qp(P, q, G, h, A, b)
        self.alphas = np.array(sol["x"])

    def predict(self, X):
        y_predict = np.zeros((X.shape[0]))
        sv = self.get_parameters(self.alphas)

        for i in range(X.shape[0]):
            y_predict[i] = np.sum(
                self.alphas[sv]
                * self.y[sv, np.newaxis]
                * self.kernel(X[i], self.X[sv])[:, np.newaxis]
            )

        return np.sign(y_predict + self.b)

    def get_parameters(self, alphas):
        threshold = 1e-5

        sv = ((alphas > threshold) * (alphas < self.C)).flatten()
        self.w = np.dot(X[sv].T, alphas[sv] * self.y[sv, np.newaxis])
        self.b = np.mean(
            self.y[sv, np.newaxis]
            - self.alphas[sv] * self.y[sv, np.newaxis] * self.K[sv, sv][:, np.newaxis]
        )
        return sv


In [ ]:

y=y.ravel()
split=int(0.1*len(X))
X_train=X[:split]
X_test=X[split:]
y_train=y[:split]
y_test=y[split:]

from sklearn.svm import SVC
svm_model = SVC(kernel='linear')
svm_model.fit(X_train,y_train)
y_predict=svm_model.predict(X_train)



**implementing the kernel trick using sklearn, this code was ran on kaggle**

In [ ]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear')  
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
y_pred = svm_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score
print("Accuracy : ", accuracy_score(y_test, y_pred))
print("precision : ", precision_score(y_test, y_pred, average="macro",zero_division=0))

Accuracy :  0.9960838272650296
precision :  0.847179928489522
